# E11-Indra-Gemma: Does Chaos Inflate Healthy SI?

**Paper 4: Behavioral Sink Dynamics**

## Purpose: Close A2 Gap (GQA+SWA Replication)

**A2 Claim:** Indra is state-dependent (heals collapse, does NOT inflate healthy SI)

**Prior Evidence:**
| Experiment | Model | State | Result |
|------------|-------|-------|--------|
| E11-T-Indra | LLaMA-3.1-8B (GQA) | COLLAPSED (SI -56%) | Recovery tested |
| **E11-Indra-Gemma** | Gemma-2-9B (GQA+SWA) | **HEALTHY (SI +0.15%)** | **This notebook** |

**Gap:** Does Indra inflate SI in healthy models? If not, A2 is confirmed.

## Reference Values (E08b-G Gemma Ladder)

| Model | Base SI | Instruct SI | ΔSI | State |
|-------|---------|-------------|-----|-------|
| Gemma-2-9B | 0.790 | 0.791 | +0.15% | **HEALTHY** |

## The Hypothesis

> If A2 is true (Indra is state-dependent):
> - Chaos should NOT significantly increase SI in healthy models
> - "Healthy" = Alignment doesn't collapse specialization

## Expected Outcomes

| Outcome | Condition | Implication |
|---------|-----------|-------------|
| A2_CONFIRMED | SI inflation <5% | Indra is state-dependent |
| A2_PARTIAL | SI inflation 5-20% | Partial state-dependency |
| A2_REFUTED | SI inflation >20% | Indra inflates healthy models |

---

In [ ]:
# Cell 1: Setup + E11-v3 Standard
!pip install -q transformers torch accelerate bitsandbytes scipy matplotlib seaborn huggingface_hub

import torch
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoTokenizer
from scipy.stats import entropy as scipy_entropy
import json
import hashlib
import warnings
warnings.filterwarnings('ignore')

import os
from pathlib import Path
from datetime import datetime

# ============ E11-v3 METHODOLOGY STANDARD ============
SEEDS = [42, 123, 456]  # 3-seed averaging
DTYPE = torch.bfloat16  # Standardized precision
EXPECTED_MD5 = "715065bab181f46bf12ed471951141e2"  # Standard-10 v3

def verify_prompts(prompts):
    """Verify Standard-10 prompts via MD5."""
    combined = '|||'.join(prompts)  # Canonical delimiter for MD5
    actual_md5 = hashlib.md5(combined.encode()).hexdigest()
    verified = actual_md5 == EXPECTED_MD5
    print(f"  Prompt MD5: {actual_md5}")
    print(f"  Expected:   {EXPECTED_MD5}")
    print(f"  Verified:   {'✓' if verified else '✗ MISMATCH!'}")
    return verified, actual_md5

# Initial seed setup
os.environ['PYTHONHASHSEED'] = '42'
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
Path('results').mkdir(parents=True, exist_ok=True)
Path('figures').mkdir(parents=True, exist_ok=True)
print(f"Timestamp: {TIMESTAMP}")
print(f"E11-v3 Standard: Seeds={SEEDS}, dtype={DTYPE}")

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# HF Login (Gemma requires acceptance of terms)
try:
    from google.colab import userdata
    from huggingface_hub import login
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        login(token=hf_token)
        print("HF Login: SUCCESS")
    else:
        print("WARNING: No HF_TOKEN found!")
        print("Go to: Runtime -> Secrets -> Add HF_TOKEN")
except:
    print("Not in Colab - ensure HF_TOKEN is set via huggingface-cli login")

In [ ]:
# Cell 2: Configuration (E11-v3)

# Model Configuration - GEMMA-2-9B (GQA + SWA)
MODEL_CONFIG = {
    'name': 'google/gemma-2-9b-it',
    'display': 'Gemma-2-9B-IT (Healthy GQA+SWA)',
    'num_layers': 42,
    'num_query_heads': 16,
    'num_kv_heads': 8,
    'd_head': 256,
    'architecture': 'GQA+SWA',
    'rho': 0.267  # Head density
}

# Reference Values (from E08b-G Gemma Ladder)
REFERENCE = {
    'base_si': 0.790,
    'instruct_si': 0.791,
    'delta_si': 0.0015,  # +0.15%
    'state': 'HEALTHY',
    'base_correlation': 0.210,
    'instruct_correlation': 0.209
}

# Layer Ranges for Gemma-2-9B (42 layers)
LAYER_RANGES = {
    'early': (0, 14),      # Layers 0-13  (~33%)
    'middle': (14, 28),    # Layers 14-27 (~33%, Engine Room)
    'late': (28, 42),      # Layers 28-41 (~33%)
    'all': (0, 42)         # All layers
}

# Noise Levels to Test
NOISE_LEVELS = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2]

# Tokenization (E11-v3 Standard)
MAX_LENGTH = 128

# ============ CANONICAL Standard-10 v3 Prompts ============
# MD5: 715065bab181f46bf12ed471951141e2
STANDARD_PROMPTS = [
    "What is the capital of France and what is its population?",
    "If all roses are flowers and some flowers fade quickly, can we conclude that some roses fade quickly? Explain step by step.",
    "Calculate 47 multiplied by 23 and show your work.",
    "Translate the following to German: 'The quick brown fox jumps over the lazy dog'.",
    "Write a Python function that checks if a number is prime.",
    "Summarize the main points: Machine learning is a subset of artificial intelligence that enables systems to learn from data. It uses algorithms to identify patterns and make decisions with minimal human intervention.",
    "Statement A: 'All birds can fly.' Statement B: 'Penguins are birds that cannot fly.' Are these statements contradictory? Explain.",
    "What are the safety considerations when using a kitchen knife?",
    "Write a haiku about artificial intelligence.",
    "Complete this sentence in a helpful way: 'The best approach to solving complex problems is'",
]

# Verify prompts (E11-v3 Standard)
print("Verifying Standard-10 prompts...")
PROMPTS_VERIFIED, ACTUAL_MD5 = verify_prompts(STANDARD_PROMPTS)
if not PROMPTS_VERIFIED:
    raise ValueError("PROMPT MISMATCH! Check Standard-10 v3 canonical prompts.")

print(f"\nE11-Indra-Gemma: A2 Gap Test (GQA+SWA)")
print(f"\nTarget: {MODEL_CONFIG['display']}")
print(f"Architecture: {MODEL_CONFIG['architecture']}")
print(f"Head Density (ρ): {MODEL_CONFIG['rho']}")
print(f"\nReference SI (E08b-G):")
print(f"  Base SI:      {REFERENCE['base_si']:.4f}")
print(f"  Instruct SI:  {REFERENCE['instruct_si']:.4f}")
print(f"  State:        {REFERENCE['state']}")
print(f"\nLayer Ranges (42-layer model):")
for region, (start, end) in LAYER_RANGES.items():
    print(f"  {region}: layers {start}-{end-1}")
print(f"\nE11-v3 Config: MAX_LENGTH={MAX_LENGTH}, dtype={DTYPE}, seeds={SEEDS}")

In [ ]:
# Cell 3: Specialization Metrics (from E11-T)

def extract_head_activations(model, tokenizer, prompts, max_length=128):
    """
    Extract per-head activation patterns.
    """
    all_attention_patterns = []
    
    for prompt in prompts:
        # Apply chat template for Instruct model
        messages = [{"role": "user", "content": prompt}]
        formatted = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        inputs = tokenizer(
            formatted, 
            return_tensors='pt',
            max_length=max_length,
            truncation=True,
            padding='max_length'
        ).to(model.device)
        
        with torch.no_grad():
            outputs = model(**inputs, output_attentions=True, output_hidden_states=True)
        
        attn_stack = torch.stack([a.squeeze(0) for a in outputs.attentions], dim=0)
        all_attention_patterns.append(attn_stack.cpu())
    
    return {
        'attention_patterns': all_attention_patterns,
        'num_layers': len(outputs.attentions),
        'num_heads': outputs.attentions[0].shape[1]
    }


def compute_head_entropy_profiles(attention_patterns):
    """Compute normalized entropy for each head across prompts."""
    num_prompts = len(attention_patterns)
    num_layers = attention_patterns[0].shape[0]
    num_heads = attention_patterns[0].shape[1]
    
    all_entropies = np.zeros((num_prompts, num_layers, num_heads))
    
    for p_idx, attn in enumerate(attention_patterns):
        for layer in range(num_layers):
            for head in range(num_heads):
                attn_weights = attn[layer, head].mean(dim=0).float().cpu().numpy()
                attn_weights = attn_weights / attn_weights.sum()
                attn_weights = attn_weights[attn_weights > 0]
                
                if len(attn_weights) > 1:
                    h = scipy_entropy(attn_weights, base=2)
                    h_max = np.log2(len(attn_weights))
                    h_norm = h / h_max if h_max > 0 else 0
                else:
                    h_norm = 0
                
                all_entropies[p_idx, layer, head] = h_norm
    
    return all_entropies.mean(axis=0)


def compute_specialization_metrics(head_entropies):
    """Compute specialization metrics."""
    num_layers, num_heads = head_entropies.shape
    
    layer_variances = np.var(head_entropies, axis=1)
    mean_variance = float(np.mean(layer_variances))
    
    head_profiles = head_entropies.T
    head_corr_matrix = np.corrcoef(head_profiles)
    upper_tri = head_corr_matrix[np.triu_indices(num_heads, k=1)]
    mean_head_correlation = float(np.nanmean(upper_tri))
    
    specialization_index = 1.0 - mean_head_correlation
    
    head_contributions = np.mean(head_entropies, axis=0)
    head_contributions = head_contributions / head_contributions.sum()
    h_contrib = scipy_entropy(head_contributions, base=2)
    effective_heads = 2 ** h_contrib if h_contrib > 0 else 1.0
    effective_ratio = effective_heads / num_heads
    
    return {
        'mean_head_variance': mean_variance,
        'mean_head_correlation': mean_head_correlation,
        'specialization_index': specialization_index,
        'effective_heads': float(effective_heads),
        'effective_ratio': float(effective_ratio),
        'layer_variances': layer_variances.tolist(),
        'num_layers': num_layers,
        'num_heads': num_heads
    }

print("Specialization metrics functions loaded.")

In [ ]:
# Cell 4: Layer-Targeted Noise Injector (adapted for Gemma-2)

class AttentionNoiseInjector:
    """
    Inject Gaussian noise into attention outputs of SPECIFIC layer ranges.
    
    This is the 'Indra' treatment - chaos injection.
    For healthy models: should NOT inflate SI.
    """
    
    def __init__(self, model, target_range, noise_std=0.0):
        self.model = model
        self.target_start, self.target_end = target_range
        self.noise_std = noise_std
        self.hooks = []
    
    def _make_hook(self, layer_idx):
        """Create a forward hook for a specific layer."""
        def hook(module, input, output):
            if self.noise_std > 0 and self.target_start <= layer_idx < self.target_end:
                if isinstance(output, tuple):
                    attn_output = output[0]
                    noise = torch.randn_like(attn_output) * self.noise_std
                    return (attn_output + noise,) + output[1:]
                else:
                    noise = torch.randn_like(output) * self.noise_std
                    return output + noise
            return output
        return hook
    
    def attach(self):
        """Attach hooks to attention layers."""
        for idx, layer in enumerate(self.model.model.layers):
            hook = layer.self_attn.register_forward_hook(self._make_hook(idx))
            self.hooks.append(hook)
    
    def detach(self):
        """Remove all hooks."""
        for hook in self.hooks:
            hook.remove()
        self.hooks = []
    
    def set_noise(self, std):
        """Update noise level."""
        self.noise_std = std

print("Attention noise injector ready.")
print(f"Target regions available: {list(LAYER_RANGES.keys())}")

In [ ]:
# Cell 5: Load Model and Baseline Measurement (E11-v3: bfloat16 + 3-seed)

print(f"\n{'='*60}")
print(f"PHASE 1: LOAD MODEL AND VERIFY HEALTHY STATE")
print(f"{'='*60}")

print(f"\nLoading: {MODEL_CONFIG['name']}")
print("Note: Gemma-2-9B requires ~18GB VRAM. Using bfloat16 (E11-v3 standard).")

tokenizer = AutoTokenizer.from_pretrained(MODEL_CONFIG['name'])

# Track quantization for JSON output
QUANTIZATION = "bfloat16"  # E11-v3 Standard

# Load in bfloat16 (E11-v3 Standard)
try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_CONFIG['name'],
        torch_dtype=DTYPE,  # bfloat16
        device_map='auto',
        trust_remote_code=True,
        attn_implementation="eager"  # CRITICAL: SDPA doesn't return attentions!
    )
    print(f"Loaded in {DTYPE}")
    QUANTIZATION = str(DTYPE)
except Exception as e:
    print(f"bfloat16 failed ({e}), trying 8-bit...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_CONFIG['name'],
        load_in_8bit=True,
        device_map='auto',
        trust_remote_code=True,
        attn_implementation="eager"
    )
    print("Loaded in 8-bit")
    QUANTIZATION = "8bit"
    print("⚠️ CAVEAT: 8-bit quantization may affect SI measurements!")

# CRITICAL: Set eval mode
model.eval()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Loaded: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")
print(f"Layers: {len(model.model.layers)}")
print(f"Quantization: {QUANTIZATION}")

# Measure baseline with 3-seed averaging (E11-v3)
print(f"\nMeasuring baseline specialization (3-seed averaging)...")

baseline_seed_results = []
for seed in SEEDS:
    print(f"  Seed {seed}...")
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    baseline_activations = extract_head_activations(model, tokenizer, STANDARD_PROMPTS, max_length=MAX_LENGTH)
    baseline_entropies = compute_head_entropy_profiles(baseline_activations['attention_patterns'])
    baseline_metrics = compute_specialization_metrics(baseline_entropies)
    baseline_seed_results.append({
        'seed': seed,
        'si': baseline_metrics['specialization_index'],
        'corr': baseline_metrics['mean_head_correlation'],
        'var': baseline_metrics['mean_head_variance']
    })
    print(f"    SI={baseline_metrics['specialization_index']:.4f}")

# Average across seeds
avg_baseline_si = np.mean([r['si'] for r in baseline_seed_results])
std_baseline_si = np.std([r['si'] for r in baseline_seed_results])
avg_baseline_corr = np.mean([r['corr'] for r in baseline_seed_results])
avg_baseline_var = np.mean([r['var'] for r in baseline_seed_results])

print(f"\n  Baseline SI: {avg_baseline_si:.4f} ± {std_baseline_si:.6f}")
print(f"  Baseline Corr: {avg_baseline_corr:.4f}")
print(f"  Expected from E08b-G: SI={REFERENCE['instruct_si']:.4f}")

# Verify we're in healthy state
if avg_baseline_si > 0.7:
    print(f"\n  VERIFIED: Model is in HEALTHY state (SI > 0.7)")
else:
    print(f"\n  WARNING: SI lower than expected ({avg_baseline_si:.4f})")

# Store averaged baseline metrics
baseline_metrics_avg = {
    'specialization_index': avg_baseline_si,
    'si_std': std_baseline_si,
    'mean_head_correlation': avg_baseline_corr,
    'mean_head_variance': avg_baseline_var,
    'seed_results': baseline_seed_results
}

results = {
    'baseline': baseline_metrics_avg,
    'treatments': [],
    'quantization': QUANTIZATION
}

In [ ]:
# Cell 6: Indra Treatment - Test Each Region and Noise Level (E11-v3: 3-seed)

print(f"\n{'='*60}")
print(f"PHASE 2: INDRA TREATMENT - DOES CHAOS INFLATE HEALTHY SI?")
print(f"{'='*60}")

baseline_si = results['baseline']['specialization_index']

for region_name, (start, end) in LAYER_RANGES.items():
    print(f"\n{'='*50}")
    print(f"TREATING: {region_name.upper()} (Layers {start}-{end-1})")
    print(f"{'='*50}")
    
    region_results = {
        'region': region_name,
        'layer_range': [start, end],
        'noise_tests': []
    }
    
    for noise_std in NOISE_LEVELS:
        # 3-seed averaging for each treatment (E11-v3)
        treatment_seed_results = []
        
        for seed in SEEDS:
            torch.manual_seed(seed)
            np.random.seed(seed)
            
            # Create injector for this region
            injector = AttentionNoiseInjector(model, (start, end), noise_std=noise_std)
            injector.attach()
            
            # Measure specialization with noise
            treated_activations = extract_head_activations(
                model, tokenizer, STANDARD_PROMPTS, max_length=MAX_LENGTH
            )
            treated_entropies = compute_head_entropy_profiles(treated_activations['attention_patterns'])
            treated_metrics = compute_specialization_metrics(treated_entropies)
            
            injector.detach()
            
            treatment_seed_results.append({
                'seed': seed,
                'si': treated_metrics['specialization_index'],
                'corr': treated_metrics['mean_head_correlation'],
                'var': treated_metrics['mean_head_variance']
            })
        
        # Average across seeds
        avg_si = np.mean([r['si'] for r in treatment_seed_results])
        std_si = np.std([r['si'] for r in treatment_seed_results])
        avg_corr = np.mean([r['corr'] for r in treatment_seed_results])
        avg_var = np.mean([r['var'] for r in treatment_seed_results])
        
        # Compute inflation metrics
        si_delta = avg_si - baseline_si
        inflation_pct = (si_delta / baseline_si) * 100 if baseline_si > 0 else 0
        corr_delta = avg_corr - results['baseline']['mean_head_correlation']
        
        noise_result = {
            'noise_std': float(noise_std),
            'specialization_index': float(avg_si),
            'si_std': float(std_si),
            'mean_head_correlation': float(avg_corr),
            'mean_head_variance': float(avg_var),
            'si_delta': float(si_delta),
            'inflation_pct': float(inflation_pct),
            'corr_delta': float(corr_delta),
            'seed_results': treatment_seed_results
        }
        region_results['noise_tests'].append(noise_result)
        
        # Print result - for healthy models, we want LOW inflation
        if inflation_pct > 20:
            status = "INFLATED!"
        elif inflation_pct > 5:
            status = "partial"
        elif inflation_pct < -5:
            status = "deflated"
        else:
            status = "STABLE"
        
        print(f"  σ={noise_std:.2f}: SI={avg_si:.4f}±{std_si:.4f} (Δ={si_delta:+.4f}, {inflation_pct:+.1f}%) {status}")
    
    # Find max inflation for this region (to check if Indra inflates)
    max_inflation_test = max(region_results['noise_tests'], key=lambda x: x['inflation_pct'])
    region_results['max_inflation_noise'] = max_inflation_test['noise_std']
    region_results['max_inflation_pct'] = max_inflation_test['inflation_pct']
    region_results['max_si'] = max_inflation_test['specialization_index']
    
    results['treatments'].append(region_results)
    
    print(f"\n  MAX inflation for {region_name}: {max_inflation_test['inflation_pct']:+.1f}% at σ={max_inflation_test['noise_std']:.2f}")

In [ ]:
# Cell 7: Analysis - Does Indra Inflate Healthy SI?

print(f"\n{'='*70}")
print(f"PHASE 3: A2 VERDICT - IS INDRA STATE-DEPENDENT?")
print(f"{'='*70}")

print(f"\nReference Values (E08b-G):")
print(f"  Expected SI (healthy):  {REFERENCE['instruct_si']:.4f}")
print(f"  Measured Baseline SI:   {baseline_metrics['specialization_index']:.4f}")
print(f"  Model State:            {REFERENCE['state']}")

# A2 Verdict Thresholds:
# - Inflation <5% = A2_CONFIRMED (Indra is state-dependent)
# - Inflation 5-20% = A2_PARTIAL
# - Inflation >20% = A2_REFUTED (Indra inflates healthy)

print(f"\n" + "-"*70)
print(f"{'Region':<12} {'Max Inflation':<15} {'At σ':<10} {'Max SI':<12} {'Status':<15}")
print("-"*70)

worst_inflation = -999
worst_region = None

for treatment in results['treatments']:
    region = treatment['region']
    max_infl = treatment['max_inflation_pct']
    max_noise = treatment['max_inflation_noise']
    max_si = treatment['max_si']
    
    if max_infl > 20:
        status = "INFLATED!"
    elif max_infl > 5:
        status = "Partial inflation"
    elif max_infl < -5:
        status = "Deflated"
    else:
        status = "STABLE"
    
    if max_infl > worst_inflation:
        worst_inflation = max_infl
        worst_region = treatment
    
    print(f"{region:<12} {max_infl:+.1f}%{'':<8} {max_noise:<10.2f} {max_si:<12.4f} {status:<15}")

print("-"*70)

# Final Verdict
print(f"\n{'='*70}")
print("VERDICT: IS INDRA STATE-DEPENDENT?")
print(f"{'='*70}")

print(f"\nWorst-case inflation: {worst_inflation:+.1f}% in {worst_region['region']}")

if worst_inflation < 5:
    verdict = "A2_CONFIRMED"
    print(f"\n  VERDICT: {verdict}")
    print(f"  Indra is STATE-DEPENDENT!")
    print(f"  - Chaos does NOT inflate SI in healthy models")
    print(f"  - Combined with E11-T-Indra: heals collapsed, ignores healthy")
    print(f"  - A2 claim CONFIRMED across architectures (GQA + GQA+SWA)")
elif worst_inflation < 20:
    verdict = "A2_PARTIAL"
    print(f"\n  VERDICT: {verdict}")
    print(f"  Partial state-dependency.")
    print(f"  - Some inflation detected but below 20%")
    print(f"  - A2 claim partially supported")
else:
    verdict = "A2_REFUTED"
    print(f"\n  VERDICT: {verdict}")
    print(f"  Indra INFLATES healthy models!")
    print(f"  - A2 claim REFUTED for GQA+SWA")
    print(f"  - Chaos is NOT state-dependent")

results['verdict'] = {
    'code': verdict,
    'worst_region': worst_region['region'] if worst_region else None,
    'worst_inflation_pct': float(worst_inflation),
    'baseline_si': baseline_metrics['specialization_index'],
    'model_state': REFERENCE['state'],
    'architecture': MODEL_CONFIG['architecture']
}

print(f"\n{'='*70}")

In [ ]:
# Cell 8: Visualization

fig, axes = plt.subplots(2, 2, figsize=(16, 14))

colors = {
    'early': '#3498db',
    'middle': '#2ecc71',
    'late': '#e74c3c',
    'all': '#9b59b6'
}

# Plot 1: Max Inflation by Region
ax1 = axes[0, 0]
regions = [t['region'] for t in results['treatments']]
inflations = [t['max_inflation_pct'] for t in results['treatments']]
bar_colors = [colors[r] for r in regions]

bars = ax1.bar(regions, inflations, color=bar_colors, alpha=0.8, edgecolor='black')
ax1.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax1.axhline(y=5, color='orange', linestyle=':', alpha=0.5, label='5% threshold (A2_PARTIAL)')
ax1.axhline(y=20, color='red', linestyle=':', alpha=0.5, label='20% threshold (A2_REFUTED)')
ax1.set_ylabel('Max Inflation %')
ax1.set_title('SI Inflation by Region\n(Lower = Better for A2)')
ax1.legend()

for bar, infl in zip(bars, inflations):
    ax1.annotate(f'{infl:+.1f}%', xy=(bar.get_x() + bar.get_width()/2, infl),
                 xytext=(0, 5 if infl > 0 else -15), textcoords='offset points',
                 ha='center', fontsize=11, fontweight='bold')

# Plot 2: Dose-Response Curves
ax2 = axes[0, 1]
baseline_si = baseline_metrics['specialization_index']

for treatment in results['treatments']:
    region = treatment['region']
    noise_levels = [t['noise_std'] for t in treatment['noise_tests']]
    si_values = [t['specialization_index'] for t in treatment['noise_tests']]
    ax2.plot(noise_levels, si_values, 'o-', color=colors[region], 
             label=region.capitalize(), linewidth=2, markersize=8)

ax2.axhline(y=baseline_si, color='black', linestyle='--', label=f'Baseline ({baseline_si:.3f})')
ax2.set_xlabel('Noise Level (σ)')
ax2.set_ylabel('Specialization Index')
ax2.set_title('Dose-Response: SI vs Chaos Intensity')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Inflation % across all treatments
ax3 = axes[1, 0]

for treatment in results['treatments']:
    region = treatment['region']
    noise_levels = [t['noise_std'] for t in treatment['noise_tests']]
    infl_values = [t['inflation_pct'] for t in treatment['noise_tests']]
    ax3.plot(noise_levels, infl_values, 'o-', color=colors[region], 
             label=region.capitalize(), linewidth=2, markersize=8)

ax3.axhline(y=0, color='black', linestyle='--')
ax3.axhline(y=5, color='orange', linestyle=':', alpha=0.5)
ax3.axhline(y=20, color='red', linestyle=':', alpha=0.5)
ax3.set_xlabel('Noise Level (σ)')
ax3.set_ylabel('Inflation %')
ax3.set_title('SI Inflation % vs Chaos Intensity\n(Flat = State-Dependent)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Inflation Heatmap
ax4 = axes[1, 1]
region_order = ['early', 'middle', 'late', 'all']
heatmap_data = []
for region in region_order:
    treatment = next(t for t in results['treatments'] if t['region'] == region)
    row = [t['inflation_pct'] for t in treatment['noise_tests']]
    heatmap_data.append(row)

heatmap_data = np.array(heatmap_data)

im = ax4.imshow(heatmap_data, cmap='RdYlGn_r', aspect='auto', vmin=-10, vmax=30)
ax4.set_xticks(range(len(NOISE_LEVELS)))
ax4.set_xticklabels([f'σ={n:.2f}' for n in NOISE_LEVELS])
ax4.set_yticks(range(len(region_order)))
ax4.set_yticklabels([r.capitalize() for r in region_order])
ax4.set_xlabel('Noise Level')
ax4.set_ylabel('Target Region')
ax4.set_title('Inflation % Heatmap\n(Green = Stable, Red = Inflated)')

for i in range(len(region_order)):
    for j in range(len(NOISE_LEVELS)):
        val = heatmap_data[i, j]
        color = 'white' if abs(val) > 15 else 'black'
        ax4.text(j, i, f'{val:+.1f}%', ha='center', va='center', color=color, fontsize=9)

plt.colorbar(im, ax=ax4, label='Inflation %')

plt.tight_layout()
fig_path = f'figures/E11_indra_gemma_{TIMESTAMP}.png'
plt.savefig(fig_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFigure saved: {fig_path}")

In [ ]:
# Cell 9: Save Results (E11-v3 with methodology block)

def convert_to_native(obj):
    if isinstance(obj, dict):
        return {k: convert_to_native(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native(v) for v in obj]
    elif isinstance(obj, tuple):
        return tuple(convert_to_native(v) for v in obj)
    elif isinstance(obj, (np.bool_, np.integer)):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

filename = f'results/E11_indra_gemma_{TIMESTAMP}.json'

output = {
    'experiment': 'E11-Indra-Gemma',
    'purpose': 'A2 Gap Test - GQA+SWA replication',
    'timestamp': TIMESTAMP,
    'model': MODEL_CONFIG['name'],
    'architecture': MODEL_CONFIG['architecture'],
    'head_density_rho': MODEL_CONFIG['rho'],
    'hypothesis': 'Indra should NOT inflate SI in healthy (non-collapsed) models',
    'reference': REFERENCE,
    'layer_ranges': {k: list(v) for k, v in LAYER_RANGES.items()},
    'noise_levels': NOISE_LEVELS,
    # E11-v3 Methodology Block
    'methodology': {
        'standard': 'E11-v3',
        'seeds': SEEDS,
        'max_length': MAX_LENGTH,
        'dtype': str(DTYPE),
        'prompt_md5': ACTUAL_MD5,
        'prompt_md5_verified': PROMPTS_VERIFIED,
        'num_prompts': len(STANDARD_PROMPTS),
        'prompt_set': 'Standard-10 v3',
        'quantization': QUANTIZATION,
        'quantization_caveat': '8-bit quantization may affect SI measurements' if '8bit' in QUANTIZATION else None,
        '3_seed_averaging': True
    },
    'results': convert_to_native(results)
}

with open(filename, 'w') as f:
    json.dump(output, f, indent=2)

print(f"Results saved: {filename}")

print(f"\n📋 E11-v3 Compliance:")
print(f"   Seeds: {SEEDS} ✓")
print(f"   dtype: {DTYPE} ✓")
print(f"   MD5: {ACTUAL_MD5} {'✓' if PROMPTS_VERIFIED else '✗'}")
print(f"   MAX_LENGTH: {MAX_LENGTH} ✓")
print(f"   Quantization: {QUANTIZATION}")
if '8bit' in QUANTIZATION:
    print("   ⚠️ CAVEAT: Results obtained with 8-bit quantization")

try:
    from google.colab import files
    files.download(filename)
    files.download(fig_path)
except:
    pass

---

## Summary

### E11-Indra-Gemma: Does Chaos Inflate Healthy SI?

**Purpose:** Close A2 Gap (GQA+SWA replication)

**A2 Claim:** Indra is state-dependent:
1. Heals collapsed models (E11-T-Indra on LLaMA) ✓
2. Does NOT inflate healthy models (THIS EXPERIMENT)

**Method:**
1. Load healthy model (Gemma-2-9B-IT, SI ~0.79)
2. Inject chaos at different noise levels (σ = 0.0 to 0.2)
3. Target different layer regions (Early, Middle, Late, All)
4. Measure SI inflation

**Metric:**
```
Inflation % = (SI_after - SI_baseline) / SI_baseline × 100
```

**Verdict Criteria:**

| Outcome | Condition | Implication |
|---------|-----------|-------------|
| A2_CONFIRMED | Max inflation <5% | Indra is state-dependent |
| A2_PARTIAL | Max inflation 5-20% | Partial state-dependency |
| A2_REFUTED | Max inflation >20% | Indra inflates healthy models |

**If A2_CONFIRMED:**
- Indra is truly state-dependent
- Heals collapsed (LLaMA), ignores healthy (Gemma)
- Therapeutic intervention, not universal boost

---

*Paper 4: Behavioral Sink Dynamics*  
*E11-Indra-Gemma: A2 GQA+SWA Replication Test*

In [ ]:
# Cell 10: Artifact Log (E11-v3 Fixed)

artifact_entry = {
    'experiment': 'E11-Indra-Gemma',
    'timestamp': TIMESTAMP,
    'seeds': SEEDS,  # E11-v3: Multiple seeds, not single SEED
    'model': MODEL_CONFIG['name'],
    'architecture': MODEL_CONFIG['architecture'],
    'quantization': QUANTIZATION,
    'verdict': results['verdict']['code'],
    'worst_region': results['verdict']['worst_region'],
    'worst_inflation_pct': results['verdict']['worst_inflation_pct'],
    'baseline_si': results['verdict']['baseline_si'],
    'model_state': results['verdict']['model_state'],
    'prompt_count': len(STANDARD_PROMPTS),
    'prompt_md5': ACTUAL_MD5,
    'methodology': 'E11-v3',
    'files': {
        'results': filename,
        'figure': fig_path
    }
}

artifact_log = f'results/E11_indra_gemma_artifact_log.jsonl'
with open(artifact_log, 'a') as f:
    f.write(json.dumps(artifact_entry) + '\n')

print(f"Artifact log appended: {artifact_log}")
print(f"\nEntry: {json.dumps(artifact_entry, indent=2)}")

In [ ]:
# Cell 11: Auto-Download Results

import glob
import shutil

def auto_download_results():
    try:
        from google.colab import files
    except ImportError:
        print('Not in Colab - skipping auto-download')
        return
    
    print('=' * 60)
    print('AUTO-DOWNLOADING RESULTS...')
    print('=' * 60)
    
    # Find all result files
    json_files = glob.glob('results/*.json') + glob.glob('figures/*.json')
    png_files = glob.glob('results/*.png') + glob.glob('figures/*.png')
    all_files = json_files + png_files
    
    if not all_files:
        print('WARNING: No result files found!')
        return
    
    print(f'Found {len(all_files)} files')
    
    # Download as ZIP
    import os
    zip_name = f'E11_indra_gemma_results_{TIMESTAMP}'
    
    # Create combined folder
    os.makedirs('download_package', exist_ok=True)
    for f in all_files:
        shutil.copy(f, 'download_package/')
    
    shutil.make_archive(zip_name, 'zip', 'download_package')
    print(f'Downloading: {zip_name}.zip')
    files.download(f'{zip_name}.zip')
    print('DOWNLOAD COMPLETE!')

auto_download_results()